# Chapter 04. 머신러닝 분류 문제 연습하기 2

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## 실습 가이드
1. 데이터를 다운로드하여 Colab에 불러옵니다.
> 다운로드한 데이터의 위치는 **Colab Notebooks/data/** 로 통일합니다.

2. 필요한 라이브러리는 모두 코드로 작성되어 있습니다.
3. 코드는 위에서부터 아래로 순서대로 실행합니다.
4. 전체 문제 구성은 좌측 첫 번째 아이콘을 통해 확인할 수 있습니다.

### <b> 데이터의 속성</b>
**시작 전에** 원본 데이콘 페이지에서 상세한 데이터에 대한 설명을 확인하세요 !

Reference : https://archive.ics.uci.edu/ml/datasets/default+of+credit+card+clients


### <b>학습목표</b>
- 1) Binary Classification 모델의 이해
- 2) 학습했던 다양한 머신러닝 분류모델의 이해
- 3) 분류를 위한 데이터 전처리 방법에 대한 이해
- 4) feature engineering에 대한 이해
- 5) 평가결과를 바탕으로 모델을 개선하는 방법 습득

- 출제자: 김용담 강사

## Step 1. 예측할 데이터 불러오기

### 문제 01. 데이터 불러오기

In [ ]:
# 분석에 필요한 라이브러리를 불러옵니다
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns 

import warnings
warnings.filterwarnings('ignore')

In [ ]:
base_path = '/content/drive/MyDrive/Colab Notebooks/data/default of credit card clients.xls'
data = pd.read_excel(base_path, header=1)
data

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,PAY_6,BILL_AMT1,BILL_AMT2,BILL_AMT3,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month
0,1,20000,2,2,1,24,2,2,-1,-1,-2,-2,3913,3102,689,0,0,0,0,689,0,0,0,0,1
1,2,120000,2,2,2,26,-1,2,0,0,0,2,2682,1725,2682,3272,3455,3261,0,1000,1000,1000,0,2000,1
2,3,90000,2,2,2,34,0,0,0,0,0,0,29239,14027,13559,14331,14948,15549,1518,1500,1000,1000,1000,5000,0
3,4,50000,2,2,1,37,0,0,0,0,0,0,46990,48233,49291,28314,28959,29547,2000,2019,1200,1100,1069,1000,0
4,5,50000,1,2,1,57,-1,0,-1,0,0,0,8617,5670,35835,20940,19146,19131,2000,36681,10000,9000,689,679,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29995,29996,220000,1,3,1,39,0,0,0,0,0,0,188948,192815,208365,88004,31237,15980,8500,20000,5003,3047,5000,1000,0
29996,29997,150000,1,3,2,43,-1,-1,-1,-1,0,0,1683,1828,3502,8979,5190,0,1837,3526,8998,129,0,0,0
29997,29998,30000,1,2,2,37,4,3,2,-1,0,0,3565,3356,2758,20878,20582,19357,0,0,22000,4200,2000,3100,1
29998,29999,80000,1,3,1,41,1,-1,0,0,0,-1,-1645,78379,76304,52774,11855,48944,85900,3409,1178,1926,52964,1804,1


### 문제 02. 데이터 미리보기

In [ ]:
# 데이터 크기 확인
print(data.shape)

# 데이터 일부 확인
data.head()

(30000, 25)


,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,PAY_6,BILL_AMT1,BILL_AMT2,BILL_AMT3,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month
0,1,20000,2,2,1,24,2,2,-1,-1,-2,-2,3913,3102,689,0,0,0,0,689,0,0,0,0,1
1,2,120000,2,2,2,26,-1,2,0,0,0,2,2682,1725,2682,3272,3455,3261,0,1000,1000,1000,0,2000,1
2,3,90000,2,2,2,34,0,0,0,0,0,0,29239,14027,13559,14331,14948,15549,1518,1500,1000,1000,1000,5000,0
3,4,50000,2,2,1,37,0,0,0,0,0,0,46990,48233,49291,28314,28959,29547,2000,2019,1200,1100,1069,1000,0
4,5,50000,1,2,1,57,-1,0,-1,0,0,0,8617,5670,35835,20940,19146,19131,2000,36681,10000,9000,689,679,0


### 문제 03. 데이터 확인하기

In [ ]:
# train 데이터에서 결측치가 있는 column 확인하기
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30000 entries, 0 to 29999
Data columns (total 25 columns):
 #   Column                      Non-Null Count  Dtype
---  ------                      --------------  -----
 0   ID                          30000 non-null  int64
 1   LIMIT_BAL                   30000 non-null  int64
 2   SEX                         30000 non-null  int64
 3   EDUCATION                   30000 non-null  int64
 4   MARRIAGE                    30000 non-null  int64
 5   AGE                         30000 non-null  int64
 6   PAY_0                       30000 non-null  int64
 7   PAY_2                       30000 non-null  int64
 8   PAY_3                       30000 non-null  int64
 9   PAY_4                       30000 non-null  int64
 10  PAY_5                       30000 non-null  int64
 11  PAY_6                       30000 non-null  int64
 12  BILL_AMT1                   30000 non-null  int64
 13  BILL_AMT2                   30000 non-null  int64
 14  BILL_A

## Step 2. Data Preprocessing

- 이번 데이터는 결측치가 없습니다.

- 학습을 위해서 column들을 정리하고, train-test split을 합니다.

### 문제 04. 데이터 feature matrix(X)와 target vector(y) 정의하기

In [ ]:
# data에서 X, y를 정의합니다.
X = data.drop(columns=["ID", "default payment next month"]) # ID는 학습과 전혀 상관이 없으므로 제외합니다.
y = data["default payment next month"] 

### 문제 05. train test 데이터 나누기

In [ ]:
# 학습의 평가를 위해서 데이터의 일부를 test data로 나누고, 나머지 데이터를 학습에 사용합니다.
from sklearn.model_selection import train_test_split

# 25%를 test data로 사용합니다.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

In [ ]:
# 잘 나누어졌는지 크기를 확인합니다.
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(22500, 23) (7500, 23) (22500,) (7500,)


## Step 3. Feature Engineering

- 분석에 사용할 feature들을 만들고, 학습을 위한 데이터를 준비하는 단계입니다.

- 분석할 데이터들의 scale을 통일시켜줍니다.

### 문제 06. feature scaling

In [ ]:
# StandardScaler로 변환하기
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_standard = scaler.fit_transform(X_train)
X_test_standard = scaler.transform(X_test)

In [ ]:
# 스케일 조정된 X_train 데이터 확인하기
X_train_standard

array([[ 1.40054764,  0.81528749, -1.07416971, ..., -0.24465204,
        -0.21847943, -0.01209907],
       [-0.28709316, -1.22656119,  0.19035279, ..., -0.2065813 ,
        -0.27945519, -0.28037774],
       [ 0.24988346, -1.22656119, -1.07416971, ..., -0.31035162,
        -0.3127147 , -0.2974412 ],
       ...,
       [ 2.55121183, -1.22656119, -1.07416971, ...,  0.34502278,
         2.50247406,  0.66787122],
       [-0.90078073,  0.81528749,  1.45487529, ..., -0.31035162,
        -0.3127147 , -0.2974412 ],
       [-1.20762451, -1.22656119,  0.19035279, ..., -0.31035162,
        -0.3127147 , -0.2574933 ]])

### 문제 07. feature scaling 2

In [ ]:
# feature scaling의 성능 비교를 위해 RobustScaler도 사용해봅니다.
from sklearn.preprocessing import RobustScaler

scaler = RobustScaler()
X_train_robust = scaler.fit_transform(X_train)
X_test_robust = scaler.transform(X_test)

In [ ]:
# 변경된 값 확인
X_test_robust

array([[ 0.10526316,  0.        ,  0.        , ..., -0.28736715,
        -0.25406125, -0.2464332 ],
       [-0.31578947,  0.        ,  1.        , ..., -0.40360554,
        -0.13315579,  0.        ],
       [ 0.        ,  0.        , -1.        , ..., -0.40360554,
        -0.39946738, -0.38910506],
       ...,
       [-0.05263158,  0.        ,  0.        , ..., -0.27902597,
         0.19121172,  2.29468223],
       [ 0.42105263,  0.        , -1.        , ..., -0.13722588,
        -0.04687084, -0.11958495],
       [ 1.57894737,  0.        ,  0.        , ...,  2.28709808,
         2.26364847,  2.20492866]])

### 참고: scikit-learn에서 제공하는 피처 스케일러(scaler)

- `StandardScaler`: 기본 스케일, 각 피처의 평균을 0, 표준편차를 1로 변환
- `RobustScaler`: 위와 유사하지만 평균 대신 중간값(median)과 일분위, 삼분위값(quartile)을 사용하여 이상치 영향을 최소화
- `MinMaxScaler`: 모든 피처의 최대치와 최소치가 각각 1, 0이 되도록 스케일 조정
- `Normalizer`: 피처(컬럼)이 아니라 row마다 정규화되며, 유클리드 거리가 1이 되도록 데이터를 조정하여 빠르게 학습할 수 있게 함

<p> 스케일 조정을 하는 이유는 데이터의 값이 너무 크거나 작을 때 학습이 제대로 되지 않을 수도 있기 때문입니다. 또한 스케일의 영향이 절대적인 분류기(예: knn과 같은 거리기반 알고리즘)의 경우, 스케일 조정을 필수적으로 검토해야 합니다.
    
<p> 반면 어떤 항목은 원본 데이터의 분포를 유지하는 것이 나을 수도 있습니다. 예를 들어, 데이터가 거의 한 곳에 집중되어 있는 feature를 표준화시켜 분포를 같게 만들었을 때, 작은 단위의 변화가 큰 차이를 나타내는 것처럼 학습될 수도 있습니다. 또한 스케일의 영향을 크게 받지 않는 분류기(예: 트리 기반 앙상블 알고리즘)를 사용할 경우에도 성능이 준수하게 나오거나 과대적합(overfitting)의 우려가 적다면 생략할 수도 있습니다.
    
<p> 스케일 조정시 유의해야할 점은 원본 데이터의 의미를 잃어버릴 수 있다는 것입니다. 최종적으로 답을 구하는 것이 목적이 아니라 모델의 해석이나 향후 다른 데이터셋으로의 응용이 더 중요할 때 원 피처에 대한 설명력을 잃어버린다면 모델 개선이 어려울 수도 있습니다. 이 점을 함께 고려하시면 좋겠습니다.

## Step 4. Training

- 공부한 각 머신러닝 분류 모델들을 돌려보고, 성능을 평가합니다.

- 아래 모델들이 그 대상에 해당합니다.

- Logistic Regression / Naive Bayes / Decision Tree / Random Forest / KNN / SVM / xgboost / lightgbm

### 문제 08. Logistic Regression 모델 돌려보기

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

log_reg = LogisticRegression()
log_reg.fit(X_train_standard, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

### 문제 09. Logistic Regression 모델의 정확도 평가하기

In [ ]:
pred = log_reg.predict(X_train_standard)
logreg_score = accuracy_score(y_train, pred)

print('Logistic Regression accuracy score: {0:0.4f}'.format(logreg_score))

Logistic Regression accuracy score: 0.8111


### 문제 10. Naive Bayes 모델 돌려보기

In [ ]:
from sklearn.naive_bayes import GaussianNB

nb = GaussianNB()
nb.fit(X_train_standard, y_train)

GaussianNB(priors=None, var_smoothing=1e-09)

### 문제 11. Naive Bayes 평가하기

In [ ]:
pred = nb.predict(X_train_standard)
nb_score = accuracy_score(y_train, pred)

print('Naive Bayes accuracy score: {0:0.4f}'.format(nb_score))

Naive Bayes accuracy score: 0.7351


### 문제 12. Random Forest 모델 돌려보기

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(max_depth=15, max_features=0.8)  
rf.fit(X_train_standard, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=15, max_features=0.8,
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [ ]:
pred = rf.predict(X_train_standard) 
rf_score = accuracy_score(y_train, pred)
print('Random Forest Model accuracy score : {0:0.4f}'.format(rf_score))

Random Forest Model accuracy score : 0.9105


### 문제 13. KNN Classification 모델 돌려보기

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier()
knn.fit(X_train_standard, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform')

In [ ]:
pred = knn.predict(X_train_standard) 
knn_score = accuracy_score(y_train, pred)
print('KNN classification accuracy score : {0:0.4f}'.format(knn_score))

KNN classification accuracy score : 0.8441


### 문제 14. Support Vector Machine 모델 돌려보기

In [ ]:
from sklearn.svm import SVC

svc = SVC()
svc.fit(X_train_standard, y_train)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [ ]:
pred = svc.predict(X_train_standard) 
svm_score = accuracy_score(y_train, pred)
print('Support Vector Machine accuracy score : {0:0.4f}'.format(svm_score))

Support Vector Machine accuracy score : 0.8249


### 문제 15. XGBoost 모델 돌려보기

- xgboost의 구현체가 많지만, 여기서는 sklearn version을 사용합니다.

In [ ]:
from xgboost.sklearn import XGBClassifier

xgb = XGBClassifier(max_depth=15, max_features=0.8)
xgb.fit(X_train_standard, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=15,
              max_features=0.8, min_child_weight=1, missing=None,
              n_estimators=100, n_jobs=1, nthread=None,
              objective='binary:logistic', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=1, seed=None, silent=None,
              subsample=1, verbosity=1)

### 문제 16. XGBoost 모델의 분류결과 확인하기

In [ ]:
pred = xgb.predict(X_train_standard) 
xgb_score = accuracy_score(y_train, pred)
print('XGBoost accuracy score : {0:0.4f}'.format(xgb_score))

XGBoost accuracy score : 0.9854


### 문제 17. Light GBM 모델 돌려보기

In [ ]:
from lightgbm.sklearn import LGBMClassifier

lgb = LGBMClassifier(max_depth=15, max_features=0.8)
lgb.fit(X_train_standard, y_train)

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.1, max_depth=15,
               max_features=0.8, min_child_samples=20, min_child_weight=0.001,
               min_split_gain=0.0, n_estimators=100, n_jobs=-1, num_leaves=31,
               objective=None, random_state=None, reg_alpha=0.0, reg_lambda=0.0,
               silent=True, subsample=1.0, subsample_for_bin=200000,
               subsample_freq=0)

### 문제 18. Light GBM 분류결과 확인하기

In [ ]:
pred = lgb.predict(X_train_standard) 
lgb_score = accuracy_score(y_train, pred)
print('XGBoost accuracy score : {0:0.4f}'.format(lgb_score))

XGBoost accuracy score : 0.8464


### 참고 : 부스팅(Boosting) 모델 개요


- 부스팅은 여러 트리의 적합 결과를 합하는 앙상블 알고리즘의 하나로, 이 때 sequential의 개념이 추가되어 있습니다. 즉 연속적인 weak learner, 바로 직전 weak learner의 error를 반영한 현재 weak learner를 잡겠다는 것입니다. 이 아이디어는 Gradient Boosting Model(GBM)에서 loss를 계속 줄이는 방향으로 weak learner를 잡는다는 개념으로 확장됩니다.

![boost](https://pluralsight2.imgix.net/guides/81232a78-2e99-4ccc-ba8e-8cd873625fdf_2.jpg)


- 부스팅 계열 모델은 XGBoost, LightGBM, CatBoost 등이 있습니다.


- 더 자세한 내용은 다음 Step에서 살펴보겠습니다.

### 문제 19. 성능이 가장 좋은 모델 3개를 뽑아서 RobustScaler로 성능 비교하기

In [ ]:
# RandomForest, XGBoost, LightGBM 3개의 모델로 비교 실험을 수행합니다.
rf2 = RandomForestClassifier(max_depth=15, max_features=0.8)
xgb2 = XGBClassifier(max_depth=15, max_features=0.8)
lgb2 = LGBMClassifier(max_depth=15, max_features=0.8)

rf2.fit(X_train_robust, y_train)
xgb2.fit(X_train_robust, y_train)
lgb2.fit(X_train_robust, y_train)

pred1 = rf2.predict(X_train_robust)
pred2 = xgb2.predict(X_train_robust)
pred3 = lgb2.predict(X_train_robust)

print('RandomForest accuracy score : {0:0.4f}'.format(accuracy_score(y_train, pred1)))
print('XGBoost accuracy score : {0:0.4f}'.format(accuracy_score(y_train, pred2)))
print('LightGBM accuracy score : {0:0.4f}'.format(accuracy_score(y_train, pred3)))

RandomForest accuracy score : 0.9107
XGBoost accuracy score : 0.9854
LightGBM accuracy score : 0.8448


### 문제 20. Test prediction

- 학습한 모델을 기반하여 test 데이터를 평가합니다. 


- 실제 현업에서는 서비스에 들어가는 기술을 test 데이터로 평가합니다. 마치 유저가 들어온 것처럼요.

In [ ]:
# 모델을 하나 정해서, test data에 대한 inference를 진행합니다.
final = lgb.predict(X_test_standard)
final2 = lgb2.predict(X_test_robust)

In [ ]:
# 성능 테스트
print('LightGBM with Standard Scaler accuracy score : {0:0.4f}'.format(accuracy_score(y_test, final)))
print('LightGBM with Robust Scaler accuracy score : {0:0.4f}'.format(accuracy_score(y_test, final2)))

LightGBM with Standard Scaler accuracy score : 0.8197
LightGBM with Robust Scaler accuracy score : 0.8188


In [ ]:
# 결과 파일 생성
result = pd.DataFrame({"ID":X_test.index, "target":final}).set_index("ID").sort_index()
result

,target
ID,
5,0
9,0
11,0
12,0
14,0
...,...
29970,0
29971,0
29975,0


### 문제 21. 최종 성능 정리

In [ ]:
print('Accuracy Comparisons for Binary Models\n')

print('logreg_score:', '{0:0.5f}'.format(accuracy_score(y_test, log_reg.predict(X_test_standard))))
print('nb_score:', '{0:0.5f}'.format(accuracy_score(y_test, nb.predict(X_test_standard))))
print('rf_score   :', '{0:0.5f}'.format(accuracy_score(y_test, rf.predict(X_test_standard))))
print('knn_score   :', '{0:0.5f}'.format(accuracy_score(y_test, knn.predict(X_test_standard))))
print('svm_score   :', '{0:0.5f}'.format(accuracy_score(y_test, svc.predict(X_test_standard))))
print('xgb_score   :', '{0:0.5f}'.format(accuracy_score(y_test, xgb.predict(X_test_standard))))
print('lgb_score   :', '{0:0.5f}'.format(accuracy_score(y_test, lgb.predict(X_test_standard))))

Accuracy Comparisons for Binary Models

logreg_score: 0.81053
nb_score: 0.72587
rf_score   : 0.81933
knn_score   : 0.79147
svm_score   : 0.82093
xgb_score   : 0.80733
lgb_score   : 0.81973
